In [1]:
import sys


base_dir = '/home/shtoshni/Research/state-probes'
sys.path.append(base_dir)

import torch
import argparse
import json
import itertools
import os
from os import path
import logging
from transformers import BartForConditionalGeneration, BartConfig, BartTokenizerFast

from data.alchemy.utils import int_to_word, colors
# from data_transformer import convert_to_transformer_batches
from data.alchemy.parseScone import loadData
from shtoshni_probing.config import PROBE_START, PROBE_END

from data.alchemy.utils import int_to_word, decide_translate, translate_states_to_nl
from data.alchemy.parseScone import getBatchesWithInit


In [2]:
dev_dataset, _, _ = loadData(split="dev", kind="alchemy", synthetic=False, base_dir=base_dir)

In [3]:
str(dev_dataset[0])

'<data.alchemy.parseScone.Datum object at 0x7f4010fbc8e0>'

In [4]:
tokenizer = BartTokenizerFast.from_pretrained("facebook/bart-base")

In [5]:
def convert_to_transformer_batches(
    dataset, tokenizer, batchsize=1, random=None,
    domain="alchemy",
    state_targets_type="state.NL", device='cuda', num_instances=20,
):
    state_targets_type_split = state_targets_type.split('.')
    batches = list(getBatchesWithInit(dataset, batchsize, get_subsequent_state=True))
    
    import random
    random.seed(10)
    random.shuffle(batches)
    
    
    with open(path.join(path.join(base_dir, "cloze_data"), "alchemy.txt"), "w") as f:
        for batch in batches[:100]:
            inputs, lang_targets, prev_state_targets, subsequent_state_targets, init_states = zip(*batch)

    #         print(init_states)

            init_state = translate_states_to_nl(
                    ' '.join(init_states[0]), "alchemy", isinstance(tokenizer, BartTokenizerFast))
            
            cur_state = translate_states_to_nl(
                    ' '.join(prev_state_targets[0]), "alchemy", isinstance(tokenizer, BartTokenizerFast))
#             print(init_state)

            input_str = ' '.join(inputs[0]).replace(' \n ', '. ')
#             print(input_str)

            target_str = ' '.join(lang_targets[0])
#             print(target_str)
            f.write(init_state + "\t" + input_str  + "\t" + cur_state + "\t" + target_str + "\n")
        
        
        


In [6]:
convert_to_transformer_batches(dev_dataset, tokenizer)